In [5]:
import numpy as np
import random
from typing import Any
import time

In [6]:
K = 6
M = 3
X = [random.choice([1, -1]) for _ in range(100_000_000)]

In [7]:
def thomas_algorithm(a, b, c, d, dtype):
    """
    Solves a linear system of equations with a tridiagonal matrix using the Thomas algorithm.

    Parameters:
    a (list): Lower diagonal elements (with n-1 elements, where n is the matrix size)
    b (list): Main diagonal elements (with n elements, where n is the matrix size)
    c (list): Upper diagonal elements (with n-1 elements, where n is the matrix size)
    d (list): Constants of the linear equations (with n elements, where n is the matrix size)

    Returns:
    list: Solution x values of the linear system
    """
    n = len(d)
    c_ = np.zeros(n, dtype=dtype)
    d_ = np.zeros(n, dtype=dtype)
    x = np.zeros(n, dtype=dtype)

    # Step 1: Decomposition / Forward sweep
    c_[0] = dtype(c[0] / b[0])
    d_[0] = dtype(d[0] / b[0])

    for i in range(1, n - 1):
        c_[i] = dtype(c[i] / (b[i] - a[i - 1] * c_[i - 1]))

    for i in range(1, n):
        d_[i] = dtype((d[i] - a[i - 1] * d_[i - 1]) / (b[i] - a[i - 1] * c_[i - 1]))

    # Step 2: Back substitution
    x[-1] = d_[-1]

    for i in range(n - 2, -1, -1):
        x[i] = dtype(d_[i] - c_[i] * x[i + 1])

    return x

In [47]:
def gauss(a, b, dtype):
    n = len(b)
    for k in range(0, n - 1):
        for i in range(k + 1, n):
            if a[i, k] != 0.0:
                lam = dtype(a[i, k] / a[k, k])
                a[i, k + 1 : n] = dtype(a[i, k + 1 : n] - lam * a[k, k + 1 : n])
                b[i] = dtype(b[i] - lam * b[k])

    for k in range(n - 1, -1, -1):
        b[k] = dtype((b[k] - np.dot(a[k, k + 1 : n], b[k + 1 : n])) / a[k, k])

    return b

In [48]:
def get_A(n: int, dtype: Any) -> np.ndarray:
    result = np.zeros((n, n), dtype=dtype)

    for i in range(n):
        result[i, i] = K

    for i in range(n-1):
        result[i, i+1] = dtype(1 / (1 + M + i))
        result[i+1, i+1] = dtype(K / (2 + M  + i))


    return result

In [49]:
def test(n: int, x: np.array, dtype: Any):
    A = get_A(n, dtype)
    B = np.matmul(A, x, dtype=dtype)
    x = np.array(x, dtype=dtype)
    upper_diagonal = np.diag(A, k=1)
    main_diagonal = np.diag(A)
    lower_diagonal = np.diag(A, k=-1)

    start_thomas = time.time()
    result_thomas = thomas_algorithm(lower_diagonal, main_diagonal, upper_diagonal, B, dtype)
    end_thomas = time.time()

    start_gauss = time.time()
    result_gauss = gauss(A, B, dtype)
    end_gauss = time.time()

    return np.average(abs(result_thomas - x)), end_thomas - start_thomas, np.average(abs(result_gauss - x)), end_gauss - start_gauss

In [56]:
results = [["n", "gauss-float", "gauss-double", "thomas-float", "thomas-double"]]
times = [["n", "gauss-float", "gauss-double", "thomas-float", "thomas-double",]]
for i in list(range(3, 21)) + [50, 100, 200, 300, 400, 500, 750, 1000, 1500, 2000, 2500, 5000]:
    results_float = test(i, np.float32(X[:i]), np.float32)
    results_double = test(i, np.float64(X[:i]), np.float64)

    results.append([
        i,
        results_float[2],
        results_double[2],
        results_float[0],
        results_double[0],
    ])

    times.append([
        i,
        results_float[3],
        results_double[3],
        results_float[1],
        results_double[1],
    ])

In [57]:
def print_results(r):
    sep = ","
    print(*r[0], sep=sep)

    for row in r[1:]:
        print(sep.join([str(row[0])] + [f"{item:.5e}" for item in row[1:]]))


print_results(results)

n,gauss-float,gauss-double,thomas-float,thomas-double
3,0.00000e+00,0.00000e+00,1.98682e-08,0.00000e+00
4,1.49012e-08,0.00000e+00,4.47035e-08,0.00000e+00
5,1.19209e-08,4.44089e-17,2.38419e-08,0.00000e+00
6,9.93411e-09,3.70074e-17,3.97364e-08,0.00000e+00
7,8.51495e-09,6.34413e-17,3.40598e-08,0.00000e+00
8,7.45058e-09,5.55112e-17,4.47035e-08,0.00000e+00
9,6.62274e-09,4.93432e-17,3.97364e-08,0.00000e+00
10,5.96046e-09,4.44089e-17,3.57628e-08,0.00000e+00
11,1.62558e-08,6.05576e-17,4.33488e-08,0.00000e+00
12,1.49012e-08,7.40149e-17,4.47035e-08,0.00000e+00
13,1.37549e-08,6.83214e-17,4.12648e-08,8.54018e-18
14,1.27724e-08,6.34413e-17,4.25747e-08,7.93016e-18
15,1.19209e-08,5.92119e-17,3.97364e-08,1.48030e-17
16,1.11759e-08,7.63278e-17,3.72529e-08,2.08167e-17
17,1.05185e-08,7.18380e-17,3.50616e-08,1.95922e-17
18,9.93411e-09,6.78470e-17,3.97364e-08,1.85037e-17
19,9.41126e-09,6.42761e-17,3.76450e-08,1.75298e-17
20,8.94070e-09,6.10623e-17,3.57628e-08,1.66533e-17
50,1.07288e-08,4.66294e-17,1.90735e

In [58]:
print_results(times)

n,gauss-float,gauss-double,thomas-float,thomas-double
3,6.58035e-05,2.59876e-05,2.81334e-05,1.69277e-05
4,1.10865e-04,5.35965e-04,2.19345e-05,5.98431e-05
5,6.88791e-04,3.80993e-04,3.19481e-05,2.98023e-05
6,2.17199e-04,3.00407e-05,3.50475e-05,2.28882e-05
7,1.24931e-04,4.81606e-05,3.09944e-05,2.47955e-05
8,2.26021e-04,1.10149e-04,3.38554e-05,2.88486e-05
9,1.49012e-04,8.72612e-05,3.79086e-05,3.19481e-05
10,1.46866e-04,4.81606e-05,4.10080e-05,2.98023e-05
11,2.16961e-04,1.38998e-04,4.38690e-05,3.60012e-05
12,2.22921e-04,1.35899e-04,4.91142e-05,3.62396e-05
13,2.07901e-04,6.19888e-05,5.19753e-05,3.91006e-05
14,2.93732e-04,1.90020e-04,5.50747e-05,4.91142e-05
15,4.25816e-04,1.65224e-04,6.03199e-05,9.91821e-05
16,3.03745e-04,1.83105e-04,5.91278e-05,4.50611e-05
17,3.52144e-04,1.17779e-04,6.50883e-05,4.29153e-05
18,4.35114e-04,1.80960e-04,6.38962e-05,5.38826e-05
19,4.37021e-04,1.25885e-04,6.41346e-05,5.10216e-05
20,4.26292e-04,1.41144e-04,6.77109e-05,4.67300e-05
50,2.54703e-03,4.51088e-04,1.51873e

In [54]:
print(test(10, np.float128(X[:10]), np.float128))

(1.084202172485504434e-20, 6.079673767089844e-05, 0.0, 0.00015306472778320312)
